In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

import dataset_compute
import snapshot_obj

In [ ]:
importlib.reload(dataset_compute)
importlib.reload(snapshot_obj)

In [ ]:
snap_id = 127
lcdm = snapshot_obj.Snapshot("V1_LR_fix", snap_id)
p082 = snapshot_obj.Snapshot("V1_LR_curvaton_p082_fix", snap_id)
p084 = snapshot_obj.Snapshot("V1_LR_curvaton_p084_fix", snap_id)
data = {"lcdm": {"snapshot": snapshot_obj.Snapshot("V1_LR_fix", snap_id)},
        "p082": {"snapshot": snapshot_obj.Snapshot("V1_LR_curvaton_p082_fix", snap_id)},
        "p084": {"snapshot": snapshot_obj.Snapshot("V1_LR_curvaton_p084_fix", snap_id)}}

In [ ]:
snap_id = 126
lcdm = snapshot_obj.Snapshot("V1_MR_fix", snap_id)
p082 = snapshot_obj.Snapshot("V1_MR_curvaton_p082_fix", snap_id)
data = {"lcdm": {"snapshot": lcdm},
        "p082": {"snapshot": p082}}

In [ ]:
n = 10
for entry in data.values():
    mass = entry["snapshot"].get_subhalos("Mass") \
        * units.g.to(units.Msun) * 10**(-10)
    idx_massive = np.argsort(-mass)[:n]
    entry["Mass"] = mass[idx_massive]
    entry["GroupNumber"] = entry["snapshot"]\
        .get_subhalos("GroupNumber").astype(int)[idx_massive]
    entry["SubGroupNumber"] = entry["snapshot"]\
        .get_subhalos("SubGroupNumber").astype(int)[idx_massive]
    
    stellar_mass = entry["snapshot"].get_subhalos("Stars/Mass") \
        * units.g.to(units.Msun) * 10**(-10)
    entry["StellarMass"] = stellar_mass[idx_massive]

In [ ]:
fstr = ""
for i in range(n):
    fstr += "    {:2d}.".format(i+1)
    for sim in data.keys():
        fstr += " &  {:2d} & {:2d} & {:6.2f} & {:6.2f}".format(
            data[sim]["GroupNumber"][i], data[sim]["SubGroupNumber"][i], 
            data[sim]["Mass"][i], data[sim]["StellarMass"][i])
    fstr += " \\\\\n"
print(fstr)

Just out of curiosity compare to my method:

In [ ]:
grouped_mass = dataset_compute.group_particles_by_subhalo(lcdm, "Masses")["Masses"]
masses = [np.sum(m_arr) for m_arr in grouped_mass]
print(np.sort(masses)[:10])
print(np.argsort(masses)[:10])

In [ ]:
masses = lcdm.get_subhalos("Mass")
print(np.sort(masses)[:10])
print(np.argsort(masses)[:10])